In [1]:
import sys
sys.path.insert(0,'C:/Users/owner/Documents/IE514/Project/gym-warehouse')

import gym
import gym_warehouse
import numpy as np
import copy

env = gym.make('warehouse-v0')
env.render()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html
SHAPE OF CLASS MAP:  (5, 10)


array([[[ 30,  30,  30],
        [ 30,  30,  30],
        [ 30,  30,  30],
        ..., 
        [ 30,  30,  30],
        [ 30,  30,  30],
        [ 30,  30,  30]],

       [[ 30,  30,  30],
        [255, 255, 255],
        [255, 255, 255],
        ..., 
        [255, 255, 255],
        [255, 255, 255],
        [ 30,  30,  30]],

       [[ 30,  30,  30],
        [255, 255, 255],
        [255, 255, 255],
        ..., 
        [255, 255, 255],
        [255, 255, 255],
        [ 30,  30,  30]],

       ..., 
       [[ 30,  30,  30],
        [255, 255, 255],
        [255, 255, 255],
        ..., 
        [255, 255, 255],
        [255, 255, 255],
        [ 30,  30,  30]],

       [[ 30,  30,  30],
        [255, 255, 255],
        [255, 255, 255],
        ..., 
        [255, 255, 255],
        [255, 255, 255],
        [ 30,  30,  30]],

       [[ 30,  30,  30],
        [ 30,  30,  30],
        [ 30,  30,  30],
        ..., 
        [ 30,  30,  30],
        [ 30,  30,  30],
        [ 30,  30,

In [11]:
class Robot:

    def __init__(self,location,name):
        self.location = location
        self.name = name
        self.loaded = None
        self.assignment = None
        self.sorted_orders = None
        self.next_move = (0,0)
        
    def get_closest_orders(self,orders):

        ord_dist=[]
        for i in orders:

            distance = abs(self.location[0] - i.location[0]) + abs(self.location[1] - i.location[1])
            moves = (self.location[0]-i.location[1],self.location[1]-i.location[1])
        
            ord_dist.append([i, distance])
            
        self.sorted_orders = sorted(ord_dist, key=lambda x: x[1])
        
#         if len(ord_dist)>2:
#         self.k_orders = np.argpartition(ord_dist,2,1)
#         print("Orders: ",self.sorted_orders)
        
    def step(self,dir_):
#         check if valid
        self.location = self.location + dir_
    
    def assign(self,order,override=False):
        
        if order == None and override:
            if self.assignment is not None:
                self.assignment.assignment = None
            self.assignment = order
            return True   
        
        elif order.assignment == None:
            
            if self.assignment is not None:
                self.assignment.assignment = None
                
            self.assignment = order
            order.assign(self)    
            return True
        
        elif override:
            if self.assignment is not None:
                self.assignment.assignment = None
                self.assignment = None
            self.assignment = order
            order.assignment.assignment = None
            order.assign(self)
            return True           
        
        elif self.assignment==order:
            return True
        
        return False
    
    def get_dist(self,order):
        if order is None:
            return 0
        return abs(self.location[0] - order.location[0]) + abs(self.location[1] - order.location[1])
    
    def get_direction(self,order):
        if order is None:
            return 0
        
        return ((self.location[0] - order.location[0]) , (self.location[1] - order.location[1]))
    
    def find_next_move(self):
        
        (up,right) = self.get_direction(self.assignment)
        
        if up == 0:
            temp_up = self.location[0]
        if up < 0:
            temp_up = self.location[0] + 1
        else:
            temp_up = self.location[0] - 1
        
        if right == 0:
            temp_right = self.location[1]
        if right < 0:
            temp_right = self.location[1] + 1
        else:
            temp_right = self.location[1] - 1
        
        return (temp_up,temp_right)
    
    def __str__(self):
        return f'{self.name}'
        
    def __repr__(self):
        return f'{self.name}'              
        

In [3]:
class Order:
    
    def __init__(self,location):
        self.location = location
        self.assignment = None
        
    def assign(self,robot):
        self.assignment = robot
        
    def __str__(self):
        return f'Location of Order is {self.location} and assignment is {self.assignment}'
        
    def __repr__(self):
        return f'Location of Order is {self.location} and assignment is {self.assignment}'
            

In [13]:
class Warehouse:
    
    def __init__(self,locations):
        self.robot_1_location = locations[0]
        self.robot_2_location = locations[1]
        
        self.robot_1 = Robot(self.robot_1_location,"Robot-1")
        self.robot_2 = Robot(self.robot_2_location,"Robot-2")
        
        self.orders = []
        
    def new_order(self,location):
        order = Order(location)
        self.orders.append(order)
        
    def order_by_closeness(self):
        self.robot_1.get_closest_orders(self.orders)
        self.robot_2.get_closest_orders(self.orders)
        
    def print_orders(self):
        print(self.orders)
        
    def get_cost(self):
        return self.robot_1.get_dist(self.robot_1.assignment) + self.robot_2.get_dist(self.robot_2.assignment)
    
    def check_swap(self):
        current_cost = self.get_cost()
        print("current cost: ",current_cost)
        swap_cost = self.robot_1.get_dist(self.robot_2.assignment) + self.robot_2.get_dist(self.robot_1.assignment)
        print("swap cost: ",swap_cost)
        
        
        if swap_cost < current_cost:
            self.robot_1.assign(self.robot_2.assignment,True)
            self.robot_2.assign(self.robot_1.assignment,True)
            print("swap occurred: post-swap cost ", swap_cost, " previous cost: ", current_cost)
            
        else:
            print("NO swap")
    
    
#    def boundary_condition(self, (temp_up, temp_right)):
#        
#        if temp_up < 0 or temp_up > 9:
#            return False
        
#        elif temp_right < 0 or temp_right > 4:
#            return False
        
#        else:
#            return True
    
    def check_valid_move(self, (temp1_up,temp1_right), (temp2_up, temp2_right)):
        
        #no need to check boundary conditions
        
        #invalid
        if temp1_up == temp2_up:
            print("same spot, up")
        
        elif temp1_up == self.robot_2.location[0]:
            print("robot1 trying up but hit robot2")
        
        elif temp2_up == self.robot_1.location[0]
            print("robot2 trying up but hit robot1")
        
        elif temp1_right == temp2_right:
            print("same spot, right")
        
        elif temp1_right == self.robot_2.location[1]:
            print("robot1 trying right but hit robot1")
        
        elif temp2_right == self.robot_1.location[1]:
            print("robot2 trying right but hit robot2")
        
        else:
            print("seems good")
        
    def move_robot(self):
        
        (temp1_up, temp1_right) = self.robot_1.next_move()
        (temp2_up, temp2_right) = self.robot_2.next_move()
        
        self.check_valid_move((temp1_up, temp1_right),(temp2_up, temp2_right))
    
    def get_order_locations(self):
        return [order.location for order in self.orders]


SyntaxError: invalid syntax (<ipython-input-13-5eae107ba754>, line 53)

In [5]:
warehouse = Warehouse([[0,1],[0,3]])

In [10]:
obs,rew,done,info = env.step([0,0])
obs=obs.T

# Every time I take a step, do the following to retrieve new orders
order_locations = warehouse.get_order_locations()
for i in range(obs.shape[0]):
    for j in range(obs.shape[1]):
        if [i,j] not in order_locations:
            if obs[i][j] == 1 or obs[i][j] == -2:
            
                warehouse.new_order([i,j])
warehouse.print_orders()

warehouse.order_by_closeness()
env.render()

if len(warehouse.orders)>0:
    idx_1 = 0
    idx_2 = 0
    # assign robot 1 to closest order
    assignment_1 = warehouse.robot_1.assign(warehouse.robot_1.sorted_orders[idx_1][0])
    while assignment_1 ==False and idx_1<len(warehouse.orders):
        assignment_1 = warehouse.robot_2.assign(warehouse.robot_2.sorted_orders[idx_1][0])  
        idx_1=idx_1+1
        
    # assign robot 2 to closest order
    assignment_2 = warehouse.robot_2.assign(warehouse.robot_2.sorted_orders[idx_2][0])
    idx_2+=1
    while assignment_2 == False and idx_2<len(warehouse.orders):
        assignment_2 = warehouse.robot_2.assign(warehouse.robot_2.sorted_orders[idx_2][0])  
        idx_2+=1
    
    # swap
    if len(warehouse.orders)>0:
        warehouse.check_swap()

    
    #move the robot
    
#     elif len(warehouse.orders)>0:
#         check swap of Order 1 and None

warehouse.print_orders()

ACTION IS:  [0, 0]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([1, 0]), array([3, 0])]
Number of Orders Fulfilled:  0
Reward:  [-10, -10]
Self.is_loaded :  [False, False]
[Location of Order is [7, 2] and assignment is Robot-1, Location of Order is [6, 3] and assignment is Robot-2, Location of Order is [6, 2] and assignment is None]
current cost:  13
swap cost:  15
NO swap
[Location of Order is [7, 2] and assignment is None, Location of Order is [6, 3] and assignment is Robot-2, Location of Order is [6, 2] and assignment is Robot-1]


In [7]:
warehouse.robot_1.assignment

Location of Order is [7, 2] and assignment is Robot-1

In [ ]:
warehouse.robot_2.assignment